# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846833032601                   -0.70    4.2
  2   -7.852319812399       -2.26       -1.53    1.0
  3   -7.852646036648       -3.49       -2.52    3.2
  4   -7.852646677032       -6.19       -3.33    2.5
  5   -7.852646685920       -8.05       -4.77    1.0
  6   -7.852646686727       -9.09       -5.24    4.0
  7   -7.852646686730      -11.54       -5.96    1.0
  8   -7.852646686730      -12.54       -7.35    2.0
  9   -7.852646686730      -14.45       -7.55    3.0
 10   -7.852646686730   +    -Inf       -8.50    2.0
 11   -7.852646686730   +  -14.45       -9.33    3.2
 12   -7.852646686730      -14.75      -10.68    1.0
 13   -7.852646686730   +  -14.75       -9.55    1.0
 14   -7.852646686730      -15.05       -9.35    1.0
 15   -7.852646686730      -15.05      -10.71    1.0
 16   -7.852646686730   +    -Inf      -10.24    1.0
 17   -7.852646686730   +    -Inf      -11.01 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846872360064                   -0.70           4.2
  2   -7.852525354429       -2.25       -1.62   0.80    4.8
  3   -7.852608373744       -4.08       -2.80   0.80    5.0
  4   -7.852646570374       -4.42       -3.54   0.80    7.2
  5   -7.852646683551       -6.95       -4.53   0.80    9.0
  6   -7.852646686682       -8.50       -5.13   0.80   10.5
  7   -7.852646686724      -10.38       -6.24   0.80   11.2
  8   -7.852646686730      -11.25       -7.13   0.80   13.5
  9   -7.852646686730      -13.60       -7.77   0.80   15.2
 10   -7.852646686730   +  -14.57       -9.16   0.80   16.8
 11   -7.852646686730      -14.75       -9.40   0.80   19.0
 12   -7.852646686730   +    -Inf      -10.67   0.80   19.0
 13   -7.852646686730   +  -14.75      -11.83   0.80   21.0
 14   -7.852646686730   +    -Inf      -12.69   0.80   24.2


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.378564e+01     3.321272e+00
 * time: 0.5114908218383789
     1     1.372553e+00     1.794183e+00
 * time: 0.701469898223877
     2    -1.398174e+00     2.061872e+00
 * time: 0.7245588302612305
     3    -3.895136e+00     1.817770e+00
 * time: 0.7572429180145264
     4    -5.028353e+00     1.855887e+00
 * time: 0.7898838520050049
     5    -6.784462e+00     1.163914e+00
 * time: 0.8226590156555176
     6    -7.401672e+00     4.768667e-01
 * time: 0.8551948070526123
     7    -7.640075e+00     1.488358e-01
 * time: 0.8778059482574463
     8    -7.747800e+00     1.679693e-01
 * time: 0.900399923324585
     9    -7.805826e+00     7.280267e-02
 * time: 0.9228999614715576
    10    -7.827547e+00     4.979709e-02
 * time: 0.9457440376281738
    11    -7.838794e+00     4.982341e-02
 * time: 0.9685039520263672
    12    -7.843715e+00     6.355768e-02
 * time: 0.9913318157196045
    13    -7.844865e+00     7.282403e-02
 * time: 1.014029979705

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846963667849                   -0.70    4.0
  2   -7.852335340853       -2.27       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686697                   -2.55
  2   -7.852646686730      -10.48       -5.89
  3   -7.852646686730   +    -Inf      -12.64
|ρ_newton - ρ_scf|  = 6.498931743842292e-13
|ρ_newton - ρ_scfv| = 5.977935591924417e-14
|ρ_newton - ρ_dm|   = 1.0621781695577835e-9
